In [1]:
from config import YOUTUBE_API_KEY
from googleapiclient.discovery import build
from IPython.display import JSON
import pandas as pd


Your main aim : Pull response and turn to json method and do analyze from this structure

Firstly you should look guides for python and quickstart.
https://developers.google.com/youtube/v3/quickstart/python

To analyse API : We need documentation.
- https://console.cloud.google.com/apis/dashboard?project=peak-hangout-408406
- https://developers.google.com/youtube/v3/getting-started

You can look this documentation , we will continue by reference tab by channels for an example. Overview. TRy it now.
@try method, request and response network is given. 

response = request.execute()

request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id="UC_x5XG1OV2P6uZZ5FSM9Ttw"
    )

youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    
- We will modify request according to the security rules and necesserities.



In [2]:
#each steps at try this method, just analyze your modification necesserities.
# Get credentials and create an API client
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=YOUTUBE_API_KEY) # we dont have credentials ,have API key so changed by API KEY


channel_ids = [
    "UCYO_jab_esuFRV4b17AJtAw",       #https://www.streamweasels.com/tools/youtube-channel-id-and-user-id-convertor/
    # Can add more channels
]

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)
)
response = request.execute()
print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'whyK4rD0f-He1JgFkyEXgnCBzwI', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'Nx46r6rKbQsg-U382LWp4m34m1Q', 'id': 'UCYO_jab_esuFRV4b17AJtAw', 'snippet': {'title': '3Blue1Brown', 'description': 'My name is Grant Sanderson. Videos here cover a variety of topics in math, or adjacent fields like physics and CS, all with an emphasis on visualizing the core ideas. To goal is to use animation to help elucidate and motivate otherwise tricky topics, and for difficult problems to be made simple with changes in perspective.\n\nFor more information, other projects, FAQs, and inquiries see the website: https://www.3blue1brown.com', 'customUrl': '@3blue1brown', 'publishedAt': '2015-03-03T23:11:55Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AIf8zZSApZdSBilmxjGARJbX1WdIFYXFv69aixy9FcJraA=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.co

In [3]:
JSON(response)

<IPython.core.display.JSON object>

To analyze of JSON ;
- https://www.objgen.com/json/local/rh49EaBVJ
- https://jsonformatter.org/json-editor


Define Methods by Functions

In [4]:
def get_channel_stats(youtube, channel_ids):
    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
    )

    response = request.execute()

    # loop through items in response
    for item in response["items"]:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    
    return pd.DataFrame(all_data)

In [5]:
channel_ids = get_channel_stats(youtube, channel_ids)

In [6]:
channel_ids

,channelName,subscribers,views,totalVideos,playlistId
0,3Blue1Brown,5730000,413842117,158,UUYO_jab_esuFRV4b17AJtAw


Playlist details

In [7]:
request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId="UUYO_jab_esuFRV4b17AJtAw"
    )
response = request.execute()

In [8]:
JSON(response)

<IPython.core.display.JSON object>

Method define

In [9]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id
        )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    return video_ids

In [10]:
playlist_id = "UUYO_jab_esuFRV4b17AJtAw"

video_ids = get_video_ids(youtube, playlist_id)

In [11]:
video_ids

['R1e7aHBKLnM', 'tjIOqIr80ns', 'W1gW1dHRsOw', 'p9i3cYMQtBY', 'X4jpqCu-wlA']

Method

In [12]:
def get_video_ids(youtube, playlist_id):
    video_ids = []

    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50
        )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')    

        
    return video_ids

In [13]:
playlist_id = "UUYO_jab_esuFRV4b17AJtAw"

video_ids = get_video_ids(youtube, playlist_id)

In [14]:
len(video_ids)

158